**Module 11: Cutting-Edge AI Research**

**Excercise-1**

**Title:** Federated Learning with Hybrid Symbolic-Neural Models

**Problem Statement:**
The goal is to demonstrate the implementation of a hybrid model that integrates symbolic rules (like logical operations) with a neural network-based model. This approach showcases how federated learning can be utilized where centralized data is unavailable, leveraging a combination of local data and shared neural network models.

**Steps to Follow:**

**1.	Define Symbolic Rules:** Implement symbolic rules (e.g., logical AND operation) as functions that can be integrated into a hybrid model.

**2.	Create Neural Network Model:** Define a neural network model using TensorFlow/Keras to process input data.

**3.	Implement Hybrid Model:** Construct a hybrid model class (HybridModel) that encapsulates both symbolic rules and the neural network model. This class should include methods to predict using symbolic rules, the neural network, and a combined prediction mechanism.

**4.	Generate Sample Data:** Generate sample data that corresponds to the logical AND operation for demonstration purposes.

**5.	Train Neural Network Model:** Compile and train the neural network model using the generated sample data.

**6.	Instantiate Hybrid Model:** Create an instance of the HybridModel class, initialized with the trained neural network model.

**7.	Make Predictions:** Use the hybrid model to make predictions on the sample data, combining symbolic and neural network-based predictions.


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model

# Define symbolic rules (example rules for logical AND operation)
def logical_and(x, y):
    return x and y

# Define neural network-based model
def create_neural_network_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Dense(64, activation='relu')(inputs)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)  # Output a probability for binary classification
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Define hybrid model integrating symbolic rules with neural network
class HybridModel:
    def __init__(self, neural_network_model):
        self.neural_network_model = neural_network_model

    def predict_with_symbolic_rules(self, inputs):
        # Unpack inputs assuming it's a list of two elements
        x, y = inputs
        # Apply symbolic rules (logical AND operation) as an additional constraint
        symbolic_output = logical_and(x, y)
        return symbolic_output

    def predict_with_neural_network(self, inputs):
        # Use neural network-based model to predict
        inputs = tf.convert_to_tensor([inputs])  # Convert inputs to tensor
        neural_network_output = self.neural_network_model.predict(inputs)
        return neural_network_output[0][0]  # Return the scalar prediction

    def predict(self, inputs):
        # Combine symbolic reasoning with neural network predictions
        symbolic_prediction = self.predict_with_symbolic_rules(inputs)
        neural_network_prediction = self.predict_with_neural_network(inputs)
        # Combine predictions using a weighted average (can be adjusted based on application)
        combined_prediction = 0.7 * symbolic_prediction + 0.3 * neural_network_prediction
        return combined_prediction

# Example usage
# Generate some sample data for logical AND operation
inputs = [[0, 0], [0, 1], [1, 0], [1, 1]]
targets = [logical_and(x[0], x[1]) for x in inputs]

# Create and compile the neural network-based model
neural_network_model = create_neural_network_model(input_shape=(2,))
neural_network_model.compile(optimizer='adam', loss='binary_crossentropy')

# Train the neural network-based model
neural_network_model.fit(inputs, targets, epochs=10)

# Create the hybrid model
hybrid_model = HybridModel(neural_network_model)

# Make predictions using the hybrid model
for input_data in inputs:
    prediction = hybrid_model.predict(input_data)
    print("Input:", input_data, "Prediction:", prediction)


Epoch 1/10
1/1 [==============================] - 1s 743ms/step - loss: 0.7030
Epoch 2/10
1/1 [==============================] - 0s 12ms/step - loss: 0.6955
Epoch 3/10
1/1 [==============================] - 0s 12ms/step - loss: 0.6882
Epoch 4/10
1/1 [==============================] - 0s 11ms/step - loss: 0.6810
Epoch 5/10
1/1 [==============================] - 0s 15ms/step - loss: 0.6741
Epoch 6/10
1/1 [==============================] - 0s 11ms/step - loss: 0.6676
Epoch 7/10
1/1 [==============================] - 0s 11ms/step - loss: 0.6612
Epoch 8/10
1/1 [==============================] - 0s 15ms/step - loss: 0.6548
Epoch 9/10
1/1 [==============================] - 0s 12ms/step - loss: 0.6485
Epoch 10/10
1/1 [==============================] - 0s 78ms/step
Input: [0, 0] Prediction: 0.14467410743236542
1/1 [==============================] - 0s 27ms/step
Input: [0, 1] Prediction: 0.13602737188339234
1/1 [==============================] - 0s 20ms/step
Input: [1, 0] Prediction: 0.138111126

**Explanation:**

**1.	Symbolic Rules and Neural Network Integration:** The HybridModel class integrates symbolic rules (logical AND) with a neural network model. It uses symbolic reasoning as an additional constraint and combines it with neural network predictions using a weighted average approach.

**2.	Training and Prediction:** The neural network model is trained on the sample data (inputs and targets) for the logical AND operation. The HybridModel instance then makes predictions on each input data point, demonstrating the combined approach of symbolic reasoning and neural network predictions.


This approach exemplifies how federated learning techniques can be employed in scenarios where symbolic rules and local data influence model predictions, offering flexibility and interpretability in machine learning applications.


**Excercise-2**

**Title:** Implementing and Evaluating a Few-Shot Learning Model using a Convolutional Neural Network (CNN)

**Problem Statement:**
Few-shot learning aims to enable a model to recognize new classes with very few training examples. In this implementation, we use a Convolutional Neural Network (CNN) based on ResNet-18 to perform few-shot classification. We generate synthetic data for support and query sets across multiple classes and evaluate the model's performance on a separate test set. The trained model is saved for future use or deployment.

**Steps to Follow:**

1.	Data Preparation:

    a.	Generate synthetic data for the support set, query set, and test set.

    b.	Apply necessary transformations to the data.

2.	Define the Dataset Class:

    a.	Create a custom dataset class to handle the data and apply transformations.

3.	Define the Few-Shot Learning Model:

    a.	Use a pretrained ResNet-18 model and modify its final layer to match the number of classes.

4.	Train the Model:

    a.	Train the model using the support and query sets.

    b.	Compute the loss and update the model parameters.

5.	Evaluate the Model:

    a.	Evaluate the trained model on a separate test set.

    b.	Calculate and display the test loss and accuracy.

6.	Save the Model:

    a.	Save the trained model for future use.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np

# Define the Few-Shot Learning Dataset
class FewShotDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample, label = self.data[idx], self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample, label

# Define the Few-Shot Learning Model
class FewShotModel(nn.Module):
    def __init__(self, num_classes):
        super(FewShotModel, self).__init__()
        self.base_model = torchvision.models.resnet18(pretrained=True)
        num_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        return self.base_model(x)

# Prepare the Few-Shot Learning Data
num_classes = 5
num_examples_per_class = 5
support_set_size = num_classes * num_examples_per_class
query_set_size = num_classes * num_examples_per_class

# Generate random data for the support set and query set (replace with your own dataset)
support_data = torch.randn(support_set_size, 3, 224, 224)  # Random RGB images (224x224)
query_data = torch.randn(query_set_size, 3, 224, 224)  # Random RGB images (224x224)
support_labels = torch.tensor(np.repeat(np.arange(num_classes), num_examples_per_class))
query_labels = torch.tensor(np.repeat(np.arange(num_classes), num_examples_per_class))

# Generate random data for the test set (replace with your own dataset)
test_data = torch.randn(query_set_size, 3, 224, 224)  # Random RGB images (224x224)
test_labels = torch.tensor(np.repeat(np.arange(num_classes), num_examples_per_class))

# Define transformations for data augmentation (optional)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create Few-Shot Learning Datasets and DataLoaders
support_dataset = FewShotDataset(support_data, support_labels, transform=transform)
query_dataset = FewShotDataset(query_data, query_labels, transform=transform)
test_dataset = FewShotDataset(test_data, test_labels, transform=transform)

support_loader = DataLoader(support_dataset, batch_size=num_examples_per_class, shuffle=True)
query_loader = DataLoader(query_dataset, batch_size=num_examples_per_class, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=num_examples_per_class, shuffle=True)

# Define Few-Shot Learning Model and Optimizer
model = FewShotModel(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the Few-Shot Learning Model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for support_batch, query_batch in zip(support_loader, query_loader):
        support_inputs, support_labels = support_batch
        query_inputs, query_labels = query_batch

        optimizer.zero_grad()

        # Forward pass (using support set for adaptation)
        support_outputs = model(support_inputs)
        support_loss = criterion(support_outputs, support_labels)
        support_loss.backward()
        optimizer.step()

        # Evaluate on the query set
        with torch.no_grad():
            query_outputs = model(query_inputs)
            query_loss = criterion(query_outputs, query_labels)
            running_loss += query_loss.item()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(query_loader)}')

# Evaluate the Few-Shot Learning Model on Test Set
model.eval()
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for test_inputs, test_labels in test_loader:
        test_outputs = model(test_inputs)
        loss = criterion(test_outputs, test_labels)
        test_loss += loss.item()

        _, predicted = torch.max(test_outputs.data, 1)
        total += test_labels.size(0)
        correct += (predicted == test_labels).sum().item()

print(f'Test Loss: {test_loss / len(test_loader)}')
print(f'Test Accuracy: {100 * correct / total}%')

# Save the Few-Shot Learning Model
model_path = 'few_shot_model.pth'
torch.save(model.state_dict(), model_path)
print(f'Model saved to {model_path}')


Epoch [1/10], Loss: 1.6576904773712158
Epoch [2/10], Loss: 1.682487463951111
Epoch [3/10], Loss: 1.6930868864059447
Epoch [4/10], Loss: 1.714703917503357
Epoch [5/10], Loss: 1.6887953281402588
Epoch [6/10], Loss: 1.6653315782546998
Epoch [7/10], Loss: 1.6665464162826538
Epoch [8/10], Loss: 1.6636360168457032
Epoch [9/10], Loss: 1.6761986494064331
Epoch [10/10], Loss: 1.6884828805923462
Test Loss: 1.8628565311431884
Test Accuracy: 12.0%
Model saved to few_shot_model.pth


**Explanation of Steps:**

1.	Data Preparation:

    a.	Generated synthetic data for support, query, and test sets.

    b.	Applied transformations for resizing and normalization.

2.	Dataset Class:

    a.	Created a FewShotDataset class to handle the data and apply transformations.

3.	Few-Shot Learning Model:

    a.	Defined a model using a pretrained ResNet-18 and modified its final layer.

4.	Training:

    a.	Trained the model using support and query sets, computed loss, and updated model parameters.

5.	Evaluation:

    a.	Evaluated the model on a separate test set, computed test loss, and accuracy.

6.	Saving:

    a.	Saved the trained model using torch.save for future use or deployment.
    This script covers the entire process from data preparation to model training, evaluation, and saving.
